## RE and etc

In [22]:
# !pip install ../../tools/whl/pymongo-4.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

In [1]:
# import pymongo
import datetime

In [2]:
import glob
import pandas as pd
import os
import joblib
import json
import pickle

In [3]:
def d2j(d, f):
    with open(f, mode="wt", encoding="utf-8") as file:
        json.dump(d, file, ensure_ascii=False, indent=2)

In [4]:
def d2p(d, f):
    with open(f, mode="wb") as handle:
        pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def p2d(f):
    with open(f, "rb") as handle:
        d = pickle.load(handle)

In [5]:
nw = datetime.datetime.now()
ts = nw.strftime("%Y%m%d_%H%M%S")
# ts = ""
print(ts)

20230829_025508


In [6]:
# inc_list = ["IY", "HO","KO","CD", "SI", "SY", "SB", "IR", "CZ", "SH"]
inc_list = ["iy", "ho", "ko", "cd", "si","to", "sy", "sb", "cz", "sh", "re"]

In [8]:
f_df = "./df_csv_raw20230828_001100.pkl"
df_org = pd.read_pickle(f_df)

df = df_org \
    .query("tbl != 're'") \
    .query("tbl in @inc_list") \
    .drop(columns = ["gp0", "gp", "file"])

dfr = df_org \
    .query("tbl == 're'")\
    .drop(columns = ["gp0", "gp", "file", "tbl"])\
    .rename(columns = {
        "f":"fre"
})
print(df.shape)
print(dfr.shape)
df.head()

(7491, 4)
(1487, 3)


,f,scheme,tbl,ym
132,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,cd,201104
133,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,cd,201104
134,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,cd,201104
135,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,cd,201105
136,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,cd,201105


In [9]:
dfr_smr = dfr \
    .groupby(["ym", "scheme"])\
    ["fre"].count()\
    .sample(frac=1, random_state=1234)\
    .reset_index()
dfr_smr.head()

,ym,scheme,fre
0,201205,med,6
1,201206,pha,4
2,202103,med,6
3,201211,pha,4
4,202012,med,6


In [10]:
dfr.head()

,fre,scheme,ym
1526,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,201104
1527,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,201105
1528,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,201106
1529,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,201107
1530,../../../work/dat/dat/hdw050/202209-018/data/d...,dpc,201108


In [11]:
%%time
l_rd = []
for r in dfr_smr.itertuples():
    rd = r._asdict()
    l_rd.append(rd)
#     print(rd)
#     rdscheme = rd["scheme"]
#     rdym = rd["ym"]
#     f_out = "./d_dicts/seq2_to_id1n_" + rdscheme + "_"+ rdym + ".pkl"
#     d_dict[(rdscheme, rdym)] = f_out
#     print(f_out)
    
#     dfr_sub = dfr.query("scheme == @rdscheme and ym == @rdym")
#     l_df = []
#     for s in dfr_sub.itertuples():
#         print(s.fre)
#         df_s = pd.read_csv(s.fre, sep=",", header=None, low_memory=False, dtype="object", usecols=[1,7])\
#             .rename(columns={
#             7:"id1n"
#         })
#         #chose seq2no and id1n
        
#         l_df.append(df_s)
#         df_s_bind = pd.concat(l_df) \
#             .drop_duplicates()
#         df_s_bind.to_pickle(f_out)
#     break

print(len(l_rd))

396
CPU times: user 3.06 ms, sys: 1.99 ms, total: 5.05 ms
Wall time: 4.89 ms


In [12]:
def rd2dict(rd):
    
#     print(rd)
    rdscheme = rd["scheme"]
    rdym = rd["ym"]
    
    # f_out = "./d_dicts/seq2_to_id1n_" + rdscheme + "_"+ rdym + ".pkl"
    f_out = "../../../share/procdat/p1_dbcreation/d_seq2_to_id1n_" + rdscheme + "_"+ rdym + ".pkl"
#     d_dict[(rdscheme, rdym)] = f_out
    
    
    dfr_sub = dfr.query("scheme == @rdscheme and ym == @rdym")
    nlen = str(len(dfr_sub))
    print(f_out + ":from_nrow " + nlen)
    d_res = dict()
    l_df = []
    for s in dfr_sub.itertuples():
#         print(s.fre)
        df_s = pd.read_csv(s.fre, sep=",", header=None, low_memory=False, dtype="object", usecols=[1,7])\
            .rename(columns={
            1:"c2",
            7:"id1n"
        })
        for t in df_s.itertuples():
            d_res[t.c2] = t.id1n
        #chose seq2no and id1n
        
#         l_df.append(df_s)
        
#     df_s_bind = pd.concat(l_df) \
#         .drop_duplicates()
#     df_s_bind.to_pickle(f_out)
    d2p(d=d_res, f=f_out)
    return (rdscheme, rdym, f_out)

In [13]:
%%time

result = joblib.Parallel(n_jobs=32, verbose = 10)(joblib.delayed(rd2dict)(rd) for rd in l_rd)

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    7.5s
[Parallel(n_jobs=32)]: Done  21 tasks      | elapsed:  3.4min
[Parallel(n_jobs=32)]: Done  34 tasks      | elapsed:  3.9min
[Parallel(n_jobs=32)]: Done  49 tasks      | elapsed:  5.4min
[Parallel(n_jobs=32)]: Done  64 tasks      | elapsed:  6.0min
[Parallel(n_jobs=32)]: Done  81 tasks      | elapsed:  8.9min
[Parallel(n_jobs=32)]: Done  98 tasks      | elapsed:  9.4min
[Parallel(n_jobs=32)]: Done 117 tasks      | elapsed: 11.1min
[Parallel(n_jobs=32)]: Done 136 tasks      | elapsed: 12.5min
[Parallel(n_jobs=32)]: Done 157 tasks      | elapsed: 14.7min
[Parallel(n_jobs=32)]: Done 178 tasks      | elapsed: 16.9min
[Parallel(n_jobs=32)]: Done 201 tasks      | elapsed: 18.3min
[Parallel(n_jobs=32)]: Done 224 tasks      | elapsed: 20.5min
[Parallel(n_jobs=32)]: Done 249 tasks      | elapsed: 22.9min
[Parallel(n_jobs=32)]: Done 274 tasks      | elapsed: 2

CPU times: user 1.57 s, sys: 421 ms, total: 2 s
Wall time: 43min 52s


[Parallel(n_jobs=32)]: Done 396 out of 396 | elapsed: 43.9min finished


In [14]:
result[:5]

[('med',
  '201205',
  '../../../share/procdat/p1_dbcreation/d_seq2_to_id1n_med_201205.pkl'),
 ('pha',
  '201206',
  '../../../share/procdat/p1_dbcreation/d_seq2_to_id1n_pha_201206.pkl'),
 ('med',
  '202103',
  '../../../share/procdat/p1_dbcreation/d_seq2_to_id1n_med_202103.pkl'),
 ('pha',
  '201211',
  '../../../share/procdat/p1_dbcreation/d_seq2_to_id1n_pha_201211.pkl'),
 ('med',
  '202012',
  '../../../share/procdat/p1_dbcreation/d_seq2_to_id1n_med_202012.pkl')]

In [15]:
d_dict = dict()

In [16]:
for t in result:
    d_dict[(t[0], t[1])] = t[2]

In [17]:
# f_d_dicts = "./d_dict_fs_dict_" + ts + ".json"
f_d_dicts = "../../../share/procdat/p1_dbcreation/d_dict_fs_dict_" + ts + ".json"
print(f_d_dicts)
d2j(d=d_dict,
    f=f_d_dicts)

../../../share/procdat/p1_dbcreation/d_dict_fs_dict_20230829_025508.json


TypeError: keys must be str, int, float, bool or None, not tuple

In [18]:
d_dict2 = dict()
d_dict2["med"] = dict()
d_dict2["dpc"] = dict()
d_dict2["pha"] = dict()

for k, v in d_dict.items():
    d_dict2[k[0]][k[1]] = v

In [19]:
# d_dict2
f_d_dicts2 = "../../../share/procdat/p1_dbcreation/d_dict_fs_dict2_" + ts + ".json"
print(f_d_dicts2)
d2j(d=d_dict2,
    f=f_d_dicts2)

../../../share/procdat/p1_dbcreation/d_dict_fs_dict2_20230829_025508.json


In [ ]:
# df_s_bind = pd.concat(l_df) \
#     .drop_duplicates()

In [41]:
d_dict2

{'med': {'201205': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201205.pkl',
  '202103': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_202103.pkl',
  '202012': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_202012.pkl',
  '201106': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201106.pkl',
  '201509': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201509.pkl',
  '201111': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201111.pkl',
  '201303': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201303.pkl',
  '202009': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_202009.pkl',
  '201807': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201807.pkl',
  '201206': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201206.pkl',
  '201312': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201312.pkl',
  '201911': '../../../strage/procdat/p1_dbcreation/d_seq2_to_id1n_med_201911.pkl

In [ ]:
# df_s_bind.shape

In [ ]:
# d_dict